# 실기시험문제

* 종양을 초음파진단기로 측정 한 결과를 환자별로 저장한 데이터셋입니다.
* 딥러닝 모델링을 통해 분류 문제(양성 : 0, 악성 : 1)를 해결하고자 합니다.


### 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adam

In [2]:
# 함수로 만들어서 사용합시다.
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err')
    plt.plot(history['val_loss'], label='val_err')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### 1.1 데이터 전처리

In [3]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/wisc_bc_data.csv"
data = pd.read_csv(path)
data.drop('id', axis = 1, inplace = True)
data['diagnosis'] = data['diagnosis'].map({'B':0, 'M':1})
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,0,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,0,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,0,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,0,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,0,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [4]:
target = 'diagnosis'
x = data.drop(target, axis = 1)
y = data.loc[:, target]
x_train, x_val, y_train, y_val = train_test_split(x, y, train_size = .3, random_state = 2022)

In [5]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## 2.실기문제1
* 아래 구조로 모델링을 설계하시오.

| Layer (type) | Output Shape |
| ---- | ---- |
| dense (Dense) | (None, 30) |
| dense_1 (Dense) | (None, 20) |
| dense_2 (Dense) | (None, 10) |
| dense_3 (Dense) | (None, 4) |
| dense_4 (Dense) | (None, 1) |

In [9]:
nfeatures = x.shape[1]


In [16]:
# 메모리 정리
clear_session()

# Sequential 타입 모델 선언
model = Sequential([Dense(30, input_shape=(nfeatures,), activation='relu'),
                    Dense(20, activation='relu'),
                    Dense(10, activation='relu'),
                    Dense(4, activation='relu'),
                    Dense(1, activation='sigmoid')])


# 모델요약
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                930       
                                                                 
 dense_1 (Dense)             (None, 20)                620       
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
 dense_3 (Dense)             (None, 4)                 44        
                                                                 
 dense_4 (Dense)             (None, 1)                 5         
                                                                 
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________


* epochs 200, batch_size = 64, validation_split 비율 20%로 학습하고
* validation set으로 예측 및 평가를 수행하시오.

In [17]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')

C:\Users\User\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [18]:
history = model.fit(x_train, y_train, 
                    epochs = 200,
                    batch_size=64,
                    validation_split=0.2).history

Epoch 1/200
3/3 [==============================] - 1s 89ms/step - loss: 0.7164 - val_loss: 0.6835
Epoch 2/200
3/3 [==============================] - 0s 20ms/step - loss: 0.7024 - val_loss: 0.6822
Epoch 3/200
3/3 [==============================] - 0s 16ms/step - loss: 0.6955 - val_loss: 0.6819
Epoch 4/200
3/3 [==============================] - 0s 19ms/step - loss: 0.6910 - val_loss: 0.6822
Epoch 5/200
3/3 [==============================] - 0s 27ms/step - loss: 0.6890 - val_loss: 0.6827
Epoch 6/200
3/3 [==============================] - 0s 25ms/step - loss: 0.6877 - val_loss: 0.6822
Epoch 7/200
3/3 [==============================] - 0s 21ms/step - loss: 0.6860 - val_loss: 0.6811
Epoch 8/200
3/3 [==============================] - 0s 22ms/step - loss: 0.6837 - val_loss: 0.6797
Epoch 9/200
3/3 [==============================] - 0s 20ms/step - loss: 0.6815 - val_loss: 0.6783
Epoch 10/200
3/3 [==============================] - 0s 19ms/step - loss: 0.6796 - val_loss: 0.6772
Epoch 11/200
3/3 [=

In [15]:
pred = model.predict(x_val)
pred = np.where(pred >= .5, 1, 0)  # 임곗값을 0.5로 주고 0.5이하는 0,이상은 1로 반환하여 예측분류.
print(classification_report(y_val, pred))

13/13 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.60      1.00      0.75       241
           1       0.00      0.00      0.00       158

    accuracy                           0.60       399
   macro avg       0.30      0.50      0.38       399
weighted avg       0.36      0.60      0.45       399



C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
